In [44]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras import backend as k 
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
import tensorflow

In [45]:
#import tensorflow as tf
#sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 15172078693267151461
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 11281553818
locality {
  bus_id: 1
  links {
  }
}
incarnation: 2657353862947673654
physical_device_desc: "device: 0, name: Tesla K80, pci bus id: 0000:00:1e.0, compute capability: 3.7"
]


In [46]:
#tbCallBack = keras.callbacks.TensorBoard(log_dir='/data/tb-logs', histogram_freq=0, write_graph=True, write_images=True)


In [47]:
img_width, img_height = 299, 299
train_data_dir = "/data/train"
validation_data_dir = "/data/train"
nb_train_samples = 602813
nb_validation_samples = 66491 

In [48]:
nr_categories = 998
batch_size = 16
epochs = 50

In [49]:
model = applications.inception_resnet_v2.InceptionResNetV2(weights = "imagenet", include_top=False, input_shape = (img_width, img_height, 3))
#from keras.models import load_model
#model = load_model("/data/vgg16_1.h5")

In [50]:
for layer in model.layers[:]:
    print(layer.name)
    layer.trainable = False

input_4
conv2d_610
batch_normalization_610
activation_610
conv2d_611
batch_normalization_611
activation_611
conv2d_612
batch_normalization_612
activation_612
max_pooling2d_13
conv2d_613
batch_normalization_613
activation_613
conv2d_614
batch_normalization_614
activation_614
max_pooling2d_14
conv2d_618
batch_normalization_618
activation_618
conv2d_616
conv2d_619
batch_normalization_616
batch_normalization_619
activation_616
activation_619
average_pooling2d_4
conv2d_615
conv2d_617
conv2d_620
conv2d_621
batch_normalization_615
batch_normalization_617
batch_normalization_620
batch_normalization_621
activation_615
activation_617
activation_620
activation_621
mixed_5b
conv2d_625
batch_normalization_625
activation_625
conv2d_623
conv2d_626
batch_normalization_623
batch_normalization_626
activation_623
activation_626
conv2d_622
conv2d_624
conv2d_627
batch_normalization_622
batch_normalization_624
batch_normalization_627
activation_622
activation_624
activation_627
block35_1_mixed
block35_1_con

In [51]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_610 (Conv2D)             (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_610 (BatchN (None, 149, 149, 32) 96          conv2d_610[0][0]                 
__________________________________________________________________________________________________
activation_610 (Activation)     (None, 149, 149, 32) 0           batch_normalization_610[0][0]    
__________________________________________________________________________________________________
conv2d_611

__________________________________________________________________________________________________
activation_786 (Activation)     (None, 8, 8, 192)    0           batch_normalization_786[0][0]    
__________________________________________________________________________________________________
conv2d_787 (Conv2D)             (None, 8, 8, 224)    129024      activation_786[0][0]             
__________________________________________________________________________________________________
batch_normalization_787 (BatchN (None, 8, 8, 224)    672         conv2d_787[0][0]                 
__________________________________________________________________________________________________
activation_787 (Activation)     (None, 8, 8, 224)    0           batch_normalization_787[0][0]    
__________________________________________________________________________________________________
conv2d_785 (Conv2D)             (None, 8, 8, 192)    399360      block8_3_ac[0][0]                
__________

block8_9_ac (Activation)        (None, 8, 8, 2080)   0           block8_9[0][0]                   
__________________________________________________________________________________________________
conv2d_810 (Conv2D)             (None, 8, 8, 192)    399360      block8_9_ac[0][0]                
__________________________________________________________________________________________________
batch_normalization_810 (BatchN (None, 8, 8, 192)    576         conv2d_810[0][0]                 
__________________________________________________________________________________________________
activation_810 (Activation)     (None, 8, 8, 192)    0           batch_normalization_810[0][0]    
__________________________________________________________________________________________________
conv2d_811 (Conv2D)             (None, 8, 8, 224)    129024      activation_810[0][0]             
__________________________________________________________________________________________________
batch_norm

In [52]:
# # Since the initial layers train very general features (e.g. edges, we're not going to retrain those)
# for layer in model.layers[:12]:
#     print(layer.name)
#     layer.trainable = False

In [53]:
#Adding custom Layers 
x = model.output
x = Flatten()(x)
x = Dense(1024, activation="relu")(x)
x = Dropout(0.5)(x)
#x = Dense(1024, activation="relu")(x)
predictions = Dense(nr_categories, activation="softmax")(x)

In [54]:
model_final = Model(input = model.input, output = predictions)
#model_final = model

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py:1: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  if __name__ == '__main__':


In [55]:
model_final.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            (None, 299, 299, 3)  0                                            
__________________________________________________________________________________________________
conv2d_610 (Conv2D)             (None, 149, 149, 32) 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_610 (BatchN (None, 149, 149, 32) 96          conv2d_610[0][0]                 
__________________________________________________________________________________________________
activation_610 (Activation)     (None, 149, 149, 32) 0           batch_normalization_610[0][0]    
__________________________________________________________________________________________________
conv2d_611

In [56]:
# compile the model 
model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.SGD(lr=0.0001, momentum=0.9), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adadelta(lr=1.0, rho=0.95, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.Adagrad(lr=0.01, epsilon=None, decay=0.0), metrics=["accuracy"])
#model_final.compile(loss = "categorical_crossentropy", optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0), metrics=["accuracy"])


In [57]:
train__val_datagen = ImageDataGenerator(rescale = 1./255, validation_split=0.1)

In [58]:
#validation_datagen = ImageDataGenerator(rescale = 1./255)

In [59]:
# img_width, img_height = 128, 128
train_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        batch_size=batch_size,
        subset="training",
        class_mode='categorical')

Found 602813 images belonging to 998 classes.


In [60]:
validation_generator = train__val_datagen.flow_from_directory(
        train_data_dir,  # this is the target directory
        target_size=(img_width, img_height),  # all images will be resized to 250x250
        subset="validation",
        batch_size=batch_size,
        class_mode='categorical')

Found 66491 images belonging to 998 classes.


In [61]:
# Save the model according to the conditions  
checkpoint = ModelCheckpoint("fgcvx_inception_resnet_v2_train_dataset.h5", monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', period=1)
early = EarlyStopping(monitor='val_acc', min_delta=0, patience=5, verbose=1, mode='auto')


In [62]:
from time import time
tensorboard = TensorBoard(log_dir="logs/{}".format(time()), histogram_freq=0, write_graph=True, write_images=True)

In [63]:
#batch 16
print(1)

1


In [64]:
model_final.fit_generator(
train_generator,
#samples_per_epoch = nb_train_samples,
steps_per_epoch = nb_train_samples/batch_size,    
epochs = epochs,
validation_data = validation_generator,
#nb_val_samples = nb_validation_samples,
validation_steps = nb_validation_samples/batch_size,    
callbacks = [checkpoint, early, tensorboard])

Epoch 1/50
37676/37675 [==============================] - 19457s 516ms/step - loss: 4.3569 - acc: 0.2426 - val_loss: 4.3604 - val_acc: 0.2004

Epoch 00001: val_acc improved from -inf to 0.20037, saving model to fgcvx_inception_resnet_v2_train_dataset.h5
Epoch 2/50
37676/37675 [==============================] - 19537s 519ms/step - loss: 3.1910 - acc: 0.3817 - val_loss: 4.0077 - val_acc: 0.2391

Epoch 00002: val_acc improved from 0.20037 to 0.23905, saving model to fgcvx_inception_resnet_v2_train_dataset.h5
Epoch 3/50
37676/37675 [==============================] - 19615s 521ms/step - loss: 2.7977 - acc: 0.4343 - val_loss: 3.9757 - val_acc: 0.2434

Epoch 00003: val_acc improved from 0.23905 to 0.24339, saving model to fgcvx_inception_resnet_v2_train_dataset.h5
Epoch 4/50
26100/37675 [===================>..........] - ETA: 1:30:18 - loss: 2.5703 - acc: 0.4659

KeyboardInterrupt: 

In [ ]:
#batch 256
print("done")